<a href="https://colab.research.google.com/github/Cobord/Quantum-Computer-Things/blob/master/Open%20Systems/quantum_channels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [26]:
import tensorflow as tf
import numpy as np
from logging import ERROR
import timeit

# See https://www.tensorflow.org/tutorials/using_gpu#allowing_gpu_memory_growth
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

num_qubits_default=10
matrix_dim_default=2**num_qubits_default
lambda_depolarize_default=.1

sess = tf.Session(config=config)

# Test execution once to detect errors early.
try:
  sess.run(tf.global_variables_initializer())
except tf.errors.InvalidArgumentError:
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise

def create_state(matrix_dim):
  state_helper = tf.cast(tf.random_normal((matrix_dim, matrix_dim)),dtype=tf.complex64)
  state_helper=state_helper+tf.scalar_mul(tf.constant(1.0j,dtype=tf.complex64),tf.cast(tf.random_normal((matrix_dim, matrix_dim)),dtype=tf.complex64))
  state=tf.matmul(tf.linalg.adjoint(state_helper),state_helper)
  state=state/tf.trace(state)
  return state

def depolarize_me(state,lambda_depolarize,matrix_dim):
  depolarize_change_helper=tf.constant((1-lambda_depolarize)/matrix_dim,dtype=tf.complex64)
  tf.logging.set_verbosity(ERROR)
  depolarize_change=tf.scalar_mul(depolarize_change_helper,tf.linalg.LinearOperatorIdentity(num_rows=matrix_dim,dtype=tf.complex64).to_dense())
  depolarize=tf.scalar_mul(lambda_depolarize,state)+depolarize_change
  return depolarize

def quantum_channel_me(state,all_krause):
  first_op=True
  for krause_operator in all_krause:
    current_change=tf.matmul(krause_operator,state)
    current_change=tf.matmul(current_change,tf.linalg.adjoint(krause_operator))
    if first_op:
      new_state=current_change
      first_op=False
    else:
      new_state=new_state+current_change
  return new_state

def cpu_depolarize(matrix_dim=matrix_dim_default,lambda_depolarize=lambda_depolarize_default):
  with tf.device('/cpu:0'):
    state=create_state(matrix_dim)
    depolarizeC=depolarize_me(state,lambda_depolarize,matrix_dim)
  sess.run(depolarizeC)
  
def gpu_depolarize(matrix_dim=matrix_dim_default,lambda_depolarize=lambda_depolarize_default):
  with tf.device('/gpu:0'):
    state=create_state(matrix_dim)
    depolarizeG=depolarize_me(state,lambda_depolarize,matrix_dim)
  sess.run(depolarizeG)

def gpu_krause_test(prob,to_print=False):
  with tf.device('/gpu:0'):
    state=create_state(2)
    krause_operator_test_zero=tf.constant([[np.sqrt(1-prob),0],[0,np.sqrt(1-prob)]],dtype=tf.complex64)
    krause_operator_test_one=tf.constant([[np.sqrt(prob),0],[0,0]],dtype=tf.complex64)
    krause_operator_test_two=tf.constant([[0,0],[0,np.sqrt(prob)]],dtype=tf.complex64)
    changed_state=quantum_channel_me(state,[krause_operator_test_zero,krause_operator_test_one,krause_operator_test_two])
  if(to_print):
    print(sess.run(state))
    print(sess.run(changed_state))
  else:
    sess.run(changed_state)
  
# Runs the op several times.
my_number=10
print('Time (s) to compute \rho and \Delta_\lambda (\rho) for %d qubits. Average of %d runs.'%(num_qubits_default,my_number))
cpu_time = timeit.timeit('cpu_depolarize()', number=my_number, setup="from __main__ import cpu_depolarize")
print('CPU (s): %.5f'%(cpu_time/my_number))
gpu_time = timeit.timeit('gpu_depolarize()', number=my_number, setup="from __main__ import gpu_depolarize")
print('GPU (s): %.5f'%(gpu_time/my_number))
print('GPU vs CPU: %.5f'%(cpu_time/gpu_time))

prob=0
gpu_krause_test(prob,True)

sess.close()

ho) for 10 qubits. Average of 10 runs.
CPU (s): 0.81507
GPU (s): 0.47559
GPU vs CPU: 1.71379
[[0.62003493+2.3180393e-09j 0.44787133-1.8033318e-01j]
 [0.44787133+1.8033318e-01j 0.379965  -2.3180393e-09j]]
[[ 0.46690476-3.7300665e-10j -0.03672429+3.6880624e-01j]
 [-0.03672429-3.6880624e-01j  0.53309524+3.7300665e-10j]]
